In [ ]:
import pandas as pd

# 파일 경로
file_path = "LIN_001_IRR_WAVE.ele"

# 파일 읽기
with open(file_path, "r", encoding="utf-8", errors="ignore") as file:
    lines = file.readlines()

# 첫 번째 줄: TITLE (무시)
# 두 번째 줄: 열 이름 (변수명 포함)
header = lines[1].strip().replace('"', '').split(", ")

# 데이터 부분 (2번째 줄 이후)
data = [line.strip().split() for line in lines[2:]]

# 데이터프레임 생성
df = pd.DataFrame(data, columns=["time", "wave_elevation"])

# 데이터 타입 변환 (문자열 -> float)
df["time"] = df["time"].astype(float)
df["wave_elevation"] = df["wave_elevation"].astype(float)

# 데이터 확인
print(df.head())

# 데이터프레임 저장 (선택 사항)
df.to_csv("wave_elevation.csv", index=False)

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# 하이퍼파라미터 설정
sequence_length = 10  # 입력 시퀀스 길이
batch_size = 32
epochs = 50
learning_rate = 0.001


# 데이터셋 클래스 정의
class TimeSeriesDataset(Dataset):
    def __init__(self, data, sequence_length):
        self.data = data
        self.sequence_length = sequence_length

    def __len__(self):
        return len(self.data) - self.sequence_length

    def __getitem__(self, idx):
        x = self.data[idx:idx + self.sequence_length]
        y = self.data[idx + self.sequence_length]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)


# 데이터 준비
data_values = df["wave_elevation"].values
dataset = TimeSeriesDataset(data_values, sequence_length)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


# 트랜스포머 모델 정의
class TransformerModel(nn.Module):
    def __init__(self, input_dim, model_dim, num_heads, num_layers, dropout=0.1):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Linear(input_dim, model_dim)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=model_dim, nhead=num_heads, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(model_dim, 1)

    def forward(self, x):
        x = self.embedding(x.unsqueeze(-1))  # (batch, seq_len, model_dim)
        x = self.transformer_encoder(x)
        x = self.fc(x[:, -1, :])  # 마지막 타임스텝만 예측
        return x.squeeze(-1)


# 모델 생성
input_dim = 1
model_dim = 64
num_heads = 4
num_layers = 2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformerModel(input_dim, model_dim, num_heads, num_layers).to(device)

# 손실 함수 및 옵티마이저 정의
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 모델 학습
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in dataloader:
        x_batch, y_batch = batch
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        output = model(x_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(dataloader):.6f}")

print("Training complete.")